In [1]:
import pandas as pd
import numpy as np
import re
from re import search
import codecs
import sys

In [2]:
stop_words = [
    "از",
    "این",
    "آن",
    "به",
    "با",
    "بر",
    "برای",
    "پس",
    "تا",
    "در",
    "را",
    "که",
    "و",
]

prefixes = [
    "با",
    "بی",
    "نا"
]

postfixes = [
    "تر",
    "ترین",
    "ات",
    "ها"
]

verb_roots = [
    "گفت",
    "رفت",
    "شد"
]


def read_dataset(path, name):
    df = pd.read_excel(path + name)
    return df

def delete_punctuations(doc):
    punctuations = '،:؛؟!»«()[]"*,{.}@!?'
    edited_doc = doc.translate(str.maketrans('', '', punctuations))
    return edited_doc

def delete_stopWords(doc):
    edited_doc = doc
    for s in stop_words:
        my_regex = r"\b"+s+r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_highFrequencyWords(inverted_indexes):
    df_temp = df.copy()
    for i in inverted_indexes:
        if len(i[1])/len(df_temp) > 0.5:
            print(i[0])
            inverted_indexes.remove(i)
    return inverted_indexes

def delete_postfixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = p + r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_prefixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = r"\b" + p
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def replaceWithRoot(tokens):   
    for i in range(0, len(tokens)):
        for root in verb_roots:
            if search(root, tokens[i]):
                print("root")
                tokens[i] = tokens[i].replace(tokens[i], root)
    return tokens

def replaceArabicWords(doc):
    doc = doc.replace('ك', 'ک')
    doc = doc.replace('ئ', 'ی')
    doc = doc.replace('ي', 'ی')
    doc = doc.replace('ؤ', 'و')
    doc = doc.replace('هٔ', 'ه')
    doc = doc.replace('ة', 'ه')
    doc = doc.replace('آ', 'ا')
    doc = doc.replace('أ', 'ا')
    doc = doc.replace('إ', 'ا')
    return doc

def tokenize(df):
    content = df.content

    tokens = []
    for i in range(0, content.size):
        doc = content[i]
        # 68000 tokens
        doc = delete_punctuations(doc)
        # 50000 tokens
        # doc = delete_stopWords(doc)

        # doc = delete_postfixes(doc)

        # doc = delete_prefixes(doc)

        # doc = replaceArabicWords(doc)
        
        tokenized_doc = doc.split()
        # tokenized_doc = replaceWithRoot(tokenized_doc)

        for token in tokenized_doc:
            temp = []
            temp.append(token)
            temp.append(df.id[i])
            tokens.append(temp)
    tokens.sort()
    return tokens

def create_inverted_indexes(tokens):
    inverted_indexes = []
    doc_temp = []
    token_temp = ""
    for token in tokens:
        if token[0] == token_temp:
            doc_temp.append(token[1])
        else:
            temp = []
            temp.append(token_temp)
            doc_temp = set(doc_temp)
            temp.append(doc_temp)
            inverted_indexes.append(temp)
            doc_temp = []

        token_temp = token[0]
        # if len(doc_temp) > 0 and token[1] != doc_temp[-1]:
        doc_temp.append(token[1])
    
    # inverted_indexes = delete_highFrequencyWords(inverted_indexes)
    return inverted_indexes

def search_token(inverted_indexes, token_name):
    for token in inverted_indexes:
        if token[0] == token_name:
            return token[1]
    print("this token not exist in our database")


In [3]:
df = read_dataset("datasets/", "IR_Spring2021_ph12_7k.xlsx")


In [16]:
tokens = tokenize(df)
inverted_indexes = create_inverted_indexes(tokens)


NameError: name 'ptint' is not defined

In [17]:
search_token(inverted_indexes, "اجتماعی")

2192769


In [15]:
# print(len(inverted_indexes))
 print(inverted_indexes)
print(len(tokens))

# for i in range(5000, 5100):
#     print(inverted_indexes[i][0])
#     print(inverted_indexes[i][1])

{27}

In [6]:
import re

tweet = "I am tired! I like fruit...and milk"
s = "هی.پسر؟ چطوری تو. من، عالیی!!!!!"
s1 = "تراوت تر عالی ترین خوبتر برتر آبها"


# clean = re.sub(r"""
#                ["،:؛؟!»«()[*,{.}@!? ]+  # Accept one or more copies of punctuation
#                \ *           # plus zero or more copies of a space,
#                """,
#                " ",          # and replace it with a single space
#                s1, flags=re.VERBOSE)

postfix = [
    "تر",
    "ترین",
    "ات",
    "ها"
]

postfix = [
    "تر",
    "ترین",
    "ات",
    "ها"
]

clean = s1
for p in postfix:
    my_regex = p + r"\b"
    clean = re.sub(my_regex , "", clean)

print(s1 + "\n" + clean)

p = '،:؛؟!»«()[]"*,{.}@!?'
s = "هی.پسر؟ چطوری تو. من، عالیی!!!!!"
s1 = "بیدارترین،خوب تر،باغ ها"



# x = s.translate(str.maketrans('', '', p))
# print(x)


# for token in doc:
#     for p in punctuations:
#         if token == p:
#             token.replace("geeks", "Geeks")


تراوت تر عالی ترین خوبتر برتر آبها
تراوت  عالی  خوب بر آب


In [7]:
from nltk.stem import PorterStemmer
stemmer= PorterStemmer()
input_str = "آمدن"
# for W in input_str:
print(stemmer.stem(input_str))



آمدن


In [8]:
from re import search
tokens = ["یس","نمی رفتن", "می روی", "میگفت"]
verb_roots = ["رفت",
        "گفت"]
# my_regex = r"[a-z0-9]+$" + p + r""
# clean = re.sub(my_regex , p, clean)
# print(clean)  

for i in range(0, len(tokens)):
    for root in verb_roots:
        if search(root, tokens[i]):
            print("true")
            tokens[i] = tokens[i].replace(tokens[i], root)
            
print(tokens)


true
true
['یس', 'رفت', 'می روی', 'گفت']


In [9]:
x  = "هٔ سلام ؤ"
x = x.replace('هٔ', 'ه')
print(x)

ه سلام ؤ
